In [15]:
import pandas as pd
import requests
import sqlite3
import json
import re
from bs4 import BeautifulSoup
from pathlib import Path

In [16]:
pt_london_2025_invited = "https://fabtcg.com/en/coverage/pro-tour-london/pairings/1/"
pt_london_2025_decklists = "https://fabtcg.com/en/coverage/pro-tour-london/decklist/{}/"
pt_london_2024_results = "https://fabtcg.com/en/coverage/pro-tour-london/results/{}/"

In [23]:
def get_pairings(url, rounds_to_be_considered):
    pairings = {}

    for round_number in rounds_to_be_considered:
        print(f"ROUND {round_number}")

        if f"round {round_number}" not in pairings:
            pairings[f"round {round_number}"] = {}

        print(f"FETCHING FROM {url.format(round_number)}")

        page = requests.get(url.format(round_number))
        soup = BeautifulSoup(page.text, "html")

        player = soup.find_all("div", {"class":"tournament-coverage__player-hero-and-deck"})
        winner = soup.find_all("div", {"class":"tournament-coverage__result"})
            
        for idx, id in enumerate(player):
            table = int(idx / 2)
            player_number = (idx % 2)
            print(f"TABLE PAIRING {table}") 

            if f"table {table}" not in pairings[f"round {round_number}"]:
                pairings[f"round {round_number}"][f"table {table}"] = {"Player 1":None, "Player 2":None, "Winner":None, "Loser":None}    

            gem_id = id.find("a", href = True)

            if gem_id:
                gem_id = re.search(r"\/(\d+)", gem_id["href"]).group(1)

            if player_number == 0:
                pairings[f"round {round_number}"][f"table {table}"]["Player 1"] = gem_id
            if player_number == 1:
                pairings[f"round {round_number}"][f"table {table}"]["Player 2"] = gem_id

        for idx, id in enumerate(winner):
            print(f"TABLE {idx} WINNER")

            if re.search(r"(\d)", id.text):
                pairings[f"round {round_number}"][f"table {idx}"]["Winner"] = int(re.search(r"(\d)", id.text).group(1))
            else:
                pairings[f"round {round_number}"][f"table {idx}"]["Winner"] = 0

        for pair in pairings[f"round {round_number}"].values():
            if pair["Winner"] == 1:
                pair["Winner"] = pair["Player 1"]
                pair["Loser"] = pair["Player 2"]
            elif pair["Winner"] == 2:
                pair["Winner"] = pair["Player 2"]
                pair["Loser"] = pair["Player 1"]
            else:
                pair["Winner"] = "Draw"

    return pairings

def get_decklist(pairings, decklist_url):
    output = []
    participants = []

    for round in pairings.values():
        for table in round.values():
            if table["Player 1"] not in participants:
                participants.append(table["Player 1"])
            if table["Player 2"] not in participants:
                participants.append(table["Player 2"])

    participants = [id for id in participants if id is not None]

    for id in participants:
        url = decklist_url.format(id)

        page = requests.get(url)
        soup = BeautifulSoup(page.text, "html")

        if page.status_code == 200:
            output.append([data.text.strip() for data in soup.find_all("td")])
        else:
            output.append[id, "Unknown", "Unknown", "Unknown", "Unknown", "Unknown"]
        
    return output

def decklist_to_df(decklists):
    decklists = pd.DataFrame.from_records(decklists)
    decklists.index = [re.search(r"\((\d+)", name).group(1) for name in decklists[0]]

    participants = decklists.iloc[:,0:5].copy()
    participants = participants.rename(columns={0:"Name", 1:"Event Date", 2:"Event Name", 3:"Format", 4:"Hero"})

    decklists = decklists.drop([0,1,2,3,4], axis=1)

    decklists = pd.melt(decklists, ignore_index=False, value_name="import name")["import name"].dropna().to_frame()

    decklists[["Copies","Card"]] = decklists["import name"].str.split(" x ", expand=True)
    decklists = decklists.drop("import name", axis=1)

    return participants, decklists

def df_to_sql(dataframe, sql_table_name):
    connection = sqlite3.connect("pt_london.db")

    dataframe.to_sql(sql_table_name, connection, if_exists="replace", index=True)

In [ ]:
rounds = [number + 1 for number in range(18) if not 5 <= number <= 11]

pt_london_2025_pairings = get_pairings(pt_london_2024_results, rounds)

pairings = [[round, table, players["Player 1"], players["Player 2"], players["Winner"], players["Loser"]] for round, table in pt_london_2025_pairings.items() for table, players in table.items()]

In [ ]:
tournament_lists_json = Path("tournament_lists.json")

if tournament_lists_json.is_file():
    with open(tournament_lists_json, encoding="UTF-8") as file:
        tournament_lists = json.load(file)
else:
    tournament_lists = get_decklist(pt_london_2025_pairings, pt_london_2025_decklists)

    with open(tournament_lists_json, "w", encoding="UTF-8") as file:
        json.dump(tournament_lists, file)

tournament_lists = [record for record in tournament_lists]

In [20]:
tournament_df = decklist_to_df(tournament_lists)

participants = tournament_df[0]
decklists = tournament_df[1]
pairings = pd.DataFrame.from_records(pairings, columns=["Round", "Table", "Player 1", "Player 2", "Winner", "Loser"])

df_to_sql(participants, "participants")
df_to_sql(decklists, "decklists")
df_to_sql(pairings, "pairings")